<div>
    <table>
        <tr>
            <td>
                <center>
                    <h1>Premise Introduction</h1>
                     <a href="https://www.psi.ch/en/ta/people/romain-sacchi">Romain Sacchi</a> (PSI)
                    <br><br>
                    Duration: 1 hour 15 minutes.
                </center>
            </td>
        </tr>
    </div>

<div class="alert alert-info">
Note: we will be using <a href="https://docs.brightway.dev/en/latest/content/installation/index.html">Brightway 2</a>, not <a href="https://docs.brightway.dev/en/legacy/">Brightway 2.5</a>, because we would like to visualize our results in <a href="https://github.com/LCA-ActivityBrowser/activity-browser">Activity Browser</a>, which is for now only compatible with Brightway 2. But if you do not require ``activity-browser``, you can run this notebook in an environment containing ``brightway25``.
</div>

In [14]:
from premise import __version__
__version__

(2, 1, 0)

In [4]:
from premise import *
import bw2data as bd

Link to Premise scenario dashboard: https://premisedash-6f5a0259c487.herokuapp.com/

Premise documentation: https://premise.readthedocs.io/en/latest/

Examples notebook: https://github.com/polca/premise/blob/master/examples/examples.ipynb

In [15]:
NewDatabase?

Init signature:
NewDatabase(
    scenarios: List[dict],
    source_version: str = '3.9',
    source_type: str = 'brightway',
    key: bytes = None,
    source_db: str = None,
    source_file_path: str = None,
    additional_inventories: List[dict] = None,
    system_model: str = 'cutoff',
    system_args: dict = None,
    use_cached_inventories: bool = True,
    use_cached_database: bool = True,
    external_scenarios: list = None,
    quiet=False,
    keep_uncertainty_data=False,
    gains_scenario='CLE',
    use_absolute_efficiency=False,
) -> None
Docstring:     
Class that represents a new wurst inventory database, modified according to IAM data.

:ivar source_type: the source of the ecoinvent database. Can be `brigthway` or `ecospold`.
:vartype source_type: str
:vartype source_db: str
:ivar system_model: Can be `cutoff` (default) or `consequential`.
:vartype system_model: str
File:           c:\software\anaconda\envs\brightway\lib\site-packages\premise\new_database.py
Type:       

In [21]:
bd.projects.set_current("demo")

In [22]:
bd.databases

Databases dictionary with 2 object(s):
	biosphere3
	ecoinvent-3.9.1-cutoff

In [26]:
ndb = NewDatabase(
    scenarios=[
        {"model":"image", "pathway":"SSP2-RCP19", "year":2050},
        {"model":"remind", "pathway":"SSP2-PkBudg500", "year":2050},
    ],
    source_db="ecoinvent-3.9.1-cutoff", # <-- name of the database in the BW2 project. Must be a string.
    source_version="3.9", # <-- version of ecoinvent. Can be "3.8", "3.9" or "3.10". Must be a string.
    key='xxxxxxxxxxxxxxxxxxxxxxxxx', # <-- decryption key
    # to be requested from the library maintainers if you want ot use default scenarios included in `premise`
)

premise v.(2, 1, 0)
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-------------

In [27]:
# select sector to integrate, or all sectors. for example, updating the electricity sector: ndb.update_electricity()
ndb.update()

Processing scenarios:   0%|                     | 0/2 [03:10<?, ?it/s]

No possible datasets found for ('treatment of wastewater, average, capacity 1.1E10l/year', None) in electricity production, at wood burning power plant BRA


AssertionError: Exchanges before and after relinking are not the same: {'water, decarbonised', 'municipal solid waste', 'transport, freight, lorry, unspecified', 'chlorine, liquid', 'wood combustion power plant 20 MW', 'lubricating oil', 'waste mineral oil', None, 'sodium chloride, powder', 'ammonia, anhydrous, liquid', 'biomass, used as fuel', 'chemical, organic', 'wood ash mixture, pure'} != {'water, decarbonised', 'municipal solid waste', 'transport, freight, lorry, unspecified', 'chlorine, liquid', 'wood combustion power plant 20 MW', 'lubricating oil', 'waste mineral oil', 'sodium chloride, powder', 'ammonia, anhydrous, liquid', 'biomass, used as fuel', 'chemical, organic', 'wood ash mixture, pure'}
electricity production, at wood burning power plant|BRA

In [ ]:
ndb.write_db_to_brightway()

In [ ]:
ndb.write_superstructure_db_to_brightway()

## Incremental Databases

The class ``IncrementalDatabase`` also implementing sectoral changes in an incremental, to easily distinguish the contribution of each sector on the results.

In [ ]:
ndb = IncrementalDatabase(
    scenarios=[
        {"model":"image", "pathway":"SSP2-RCP26", "year":2040},
    ],
    source_db="ecoinvent-3.9.-cutoff", # <-- name of the database in the BW2 project. Must be a string.
    source_version="3.10", # <-- version of ecoinvent. Can be "3.5", "3.6", "3.7" or "3.8". Must be a string.
    key="xxxxxxxxxxxxxxxxxx",
)

In [ ]:
sectors = {
    "electricity": "electricity",
    "steel": "steel",
    "others": [
        "cement",
        "cars",
        "fuels"
    ]
}

In [ ]:
ndb.update(sectors=sectors)

In [ ]:
ndb.write_increment_db_to_brightway(name="test increment", file_format="csv")

## Reports
### Scenario report
You can generate a spreadsheet report showing the main variables of the scenario you have selected to create your databases. The report is saved in your working directory. Note that this report is generated automatically when exporting a database.

In [ ]:
ndb.generate_scenario_report()

## Changes report
You can generate a spreadsheet report of the changes made to the original database. It gives an overview on:
* the datasets created
* the datasets modified
* some performance indicators
* scaling factors used to scale certain exchanges

A "Validation" tab also shows any datasets that contain values or efficiencies that may seem incorrect.

The report is saved in your working directory. It is generated automatically when you export a database.

In [ ]:
ndb.generate_change_report()

## Custom scenarios

You can also create your own scenarios by providing a data package to premise. This data package should contain the following files:
* ``datapackage.json``: a json file containing the metadata of the data package
* ``scenario_data.csv``: a csv file containing the scenario data
* ``config.yaml``: a yaml file containing the configuration of the scenario (which markets to create, etc.)
* ``lci-inventories.csv``: a csv file containing additional inventories to import

In [ ]:
import brightway2 as bw
from premise import NewDatabase
from datapackage import Package


fp = r"https://raw.githubusercontent.com/premise-community-scenarios/ammonia-prospective-scenarios/main/datapackage.json"
ammonia = Package(fp)

bw.projects.set_current("your_bw_project")

ndb = NewDatabase(
    scenarios = [
        {"model":"image", "pathway":"SSP2-Base", "year":2050, "external scenarios": [{"scenario": "Business As Usual - image", "data": ammonia}]},
        {"model":"image", "pathway":"SSP2-RCP26", "year":2030, "external scenarios": [{"scenario": "Sustainable development - image", "data": ammonia}]},
    ],        
    source_db="ecoinvent 3.10 cutoff",
    source_version="3.10",
    key='xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx',
)
    
ndb.update("external") # or ndb.update() if you want to update the database with the IAM data plus the external scenario
